# Setup

In [4]:
import pandas as pd
import parse as p
import os
import altair as alt

In [29]:
import importlib
importlib.reload(p)

folder_output = './data/output'
filenames_all = sorted(os.listdir(folder_output))
filenames = [f for f in filenames_all if p.filter_monthly(f)]

df_all = p.get_full_df(filenames)
df_all = p.correct_issues(df_all)
df_all = p.add_cols(df_all)

# Analysis

## Rainfall

The raw data provides the cumulative rain in the
water-year (starting October 1) compared to a historic average since 1970.
My guess is that this format can be helpful if we assume that there's some
year-long offsetting, e.g. a large amount of early-year rain is likely followed
by less rain later on.

In [82]:
cols_reset = ['rainfallsince', 'avgrainfall1971_2000']

# Set these columns to zero on the first climatic month
# This looks like a data error, but need to investigate
df_all.loc[df_all['month_climatic'] == 1, cols_reset] = 0

cols = ['rainfallsince', 'avgrainfall1971_2000']

agg = (df_all
      .query('~ is_month_start')
      .groupby(['date', 'province'])[cols]
      .sum())

agg['relative_rainfall'] = agg[cols[0]] / agg[cols[1]]
# Limit to between -5 and 5
# agg.loc[agg.relative_rainfall < -5, 'relative_rainfall'] = -5

#if 'change' in cols[0]:
top_code = 2
agg.loc[agg.relative_rainfall > top_code, 'relative_rainfall'] = top_code
agg.loc[agg.relative_rainfall < 0, 'relative_rainfall'] = 0

title = f'Rainfall compared to historical average during climate-year (top-coded at {top_code*100}%)'
chart = alt.Chart(agg.reset_index()).mark_line().encode(
    x=alt.X('date'),
    y=alt.Y('relative_rainfall', axis=alt.Axis(format='%'), title=None),
    color='province'
).properties(
    width=600,
    height=300,
    title=title
)
rule = alt.Chart(pd.DataFrame({'y': [1]})).mark_rule(color='black').encode(y='y')
chart = rule + chart
chart

alt.LayerChart(...)

### Summary
 
Rain has been below historical averages after the 2018 water-year.

## Storage over time

We see the generally declining trend, especially since 2018. 
The large amount of rain in early 2018 goes along (and probably caused)
the increased storage. Since then, storage levels have declined relatively consistently,
from an average of 60% in 2018 towards 20% at the beginning of 2024.

In [27]:
cols = ['rainfallsince', 'stored_hm3', 'capacity_hm3']
agg = df_all.groupby(['date', 'province'])[cols].sum()
agg['share_storage'] = agg.stored_hm3 / agg.capacity_hm3

chart = alt.Chart(agg.reset_index()).mark_line().encode(
    x=alt.X('date'),
    y=alt.Y('share_storage', axis=alt.Axis(format='%'), title=None),
    color='province',
).properties(
    width=600,
    height=300
)

title = 'Fill share over time'
chart = chart.properties(title=title)

chart

alt.Chart(...)

# Appendix - various scratch code

In [69]:
group_vars = ['reservoir', 'year_climatic']
df_all = df_all.sort_values(group_vars + ['date'])
grouped = df_all.groupby(group_vars)
df_all['rainfallsince_lag'] = grouped.rainfallsince.shift(1)
df_all['avgrainfall1971_2000_lag'] = grouped.avgrainfall1971_2000.shift(1)

df_all['rainfall_change'] = df_all['rainfallsince'] - df_all['rainfallsince_lag']
df_all['avgrainfall_change'] = df_all['avgrainfall1971_2000'] - df_all['avgrainfall1971_2000_lag']

# Quick validation
cols = ['reservoir', 'date', 
        'rainfallsince', 'avgrainfall1971_2000', 
        'rainfall_change', 'avgrainfall_change', ]

df_all[cols].head(10)

,reservoir,date,rainfallsince,avgrainfall1971_2000,rainfall_change,avgrainfall_change
31,aguascebas,2012-09-01,253.4,673.4,NaN,NaN
31,aguascebas,2012-10-01,0.0,0.0,NaN,NaN
31,aguascebas,2012-11-01,6.8,88.9,6.8,88.9
31,aguascebas,2012-12-01,464.7,170.4,457.9,81.5
31,aguascebas,2013-01-01,560.9,260.0,96.2,89.6
31,aguascebas,2013-02-01,782.0,328.4,221.1,68.4
31,aguascebas,2013-03-01,888.6,395.2,106.6,66.8
31,aguascebas,2013-04-01,1319.0,481.0,430.4,85.8
31,aguascebas,2013-05-01,1464.8,562.9,145.8,81.9
31,aguascebas,2013-06-01,1554.7,627.9,89.9,65.0


In [70]:
df_all.query('reservoir=="almodovar"').query('year==2023')[cols]

,reservoir,date,rainfallsince,avgrainfall1971_2000,rainfall_change,avgrainfall_change
0,almodovar,2023-01-01,394.8,615.3,309.2,259.5
0,almodovar,2023-02-01,503.3,796.2,108.5,180.9
0,almodovar,2023-03-01,525.0,894.5,21.7,98.3
0,almodovar,2023-04-01,579.9,1007.5,54.9,113.0
0,almodovar,2023-05-01,579.9,1091.0,0.0,83.5
0,almodovar,2023-06-01,594.7,1132.4,14.8,41.4
0,almodovar,2023-07-01,610.9,1146.5,16.2,14.1
0,almodovar,2023-08-01,610.9,1147.0,0.0,0.5
0,almodovar,2023-09-01,610.9,1152.8,0.0,5.8
0,almodovar,2023-10-01,0.0,0.0,NaN,NaN


In [71]:
# Data issues: the cumulative rain declines within a water-year. Need to investigate.

df_all[df_all.rainfall_change < 0][cols].head(10)

,reservoir,date,rainfallsince,avgrainfall1971_2000,rainfall_change,avgrainfall_change
67,andevalo,2014-05-01,329.2,617.1,-55.9,56.5
67,andevalo,2014-08-01,340.6,670.2,-27.4,1.5
67,andevalo,2016-02-01,0.0,453.2,-185.0,102.6
67,andevalo,2022-02-01,0.0,453.2,-292.5,102.6
67,andevalo,2022-05-01,326.2,617.1,-148.1,56.5
67,andevalo,2022-06-01,313.4,655.2,-12.8,38.1
67,andevalo,2022-07-01,290.4,668.7,-23.0,13.5
0,arcos,2014-07-01,496.3,589.5,-10.7,9.4
3,bornos,2019-06-01,467.8,572.6,-0.6,27.1
4,celemin,2022-07-01,604.9,956.0,-0.1,12.6


In [81]:
cols = ['rainfall_change', 'avgrainfall_change']


alt.LayerChart(...)

# Additional questions

1. To what extent can the drought explain the lower fill rates? Other explanations are changes in usage patterns (e.g in agriculture) and population growth.